# Importação dos dados do Banco de Dados "clvhealth_jcafb_2025_15"

## Leitura das Credenciais

In [1]:
import sqlite3
import pandas as pd

conn = sqlite3.connect('data/jcafb_2025.db')

hosts = pd.read_sql_query('SELECT * FROM host', conn, index_col='name')

conn.close()

creds = lambda index, col_name : hosts.at[index, col_name]


## Parâmetros globais

In [2]:
import logging
# import host
import ssl

logging.basicConfig(level=logging.DEBUG)
_logger = logging.getLogger(__name__)

host_name = 'tkl-odoo15-jcafb25-vm'
# host_name = 'tkl-odoo16-vm-18'

ssl._create_default_https_context = ssl._create_unverified_context
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE


In [3]:
from xmlrpc import client
# room data with search_read method
server_url = creds(host_name, 'server')
db_name = creds(host_name, 'dbname')
username = creds(host_name, 'user')
# password = creds(host_name, 'user_pw')
password = creds(host_name, 'user_apikey')


## Importação dos dados de "res.country"

In [4]:
res_country_fields = ['id', 'name', 'code']

common = client.ServerProxy('%s/xmlrpc/2/common' % server_url)
user_id = common.authenticate(db_name, username, password, {})
models = client.ServerProxy('%s/xmlrpc/2/object' % server_url)

if user_id:

    search_domain = []
    res_country_objects = models.execute_kw(db_name, user_id, password,
                                            'res.country', 'search_read',
                                            [search_domain, res_country_fields],
                                            {}
                                           )
    res_country = pd.DataFrame(res_country_objects)

    print(res_country)

    conn = sqlite3.connect('data/jcafb_2025.db')

    cur = conn.cursor()
    cur.execute('DELETE FROM res_country')
    conn.commit()

    # res_country.to_sql('res_country', conn, if_exists='replace', index=False)
    res_country.to_sql('res_country', conn, if_exists='append', index=False)

    conn.close()


      id            name code
0      3     Afghanistan   AF
1      6         Albania   AL
2     62         Algeria   DZ
3     11  American Samoa   AS
4      1         Andorra   AD
..   ...             ...  ...
245   66  Western Sahara   EH
246  245           Yemen   YE
247  248          Zambia   ZM
248  249        Zimbabwe   ZW
249   15   Åland Islands   AX

[250 rows x 3 columns]


## Importação dos dados de "res.partner"

In [5]:
res_partner_fields = ['id', 'name', 'type']

common = client.ServerProxy('%s/xmlrpc/2/common' % server_url)
user_id = common.authenticate(db_name, username, password, {})
models = client.ServerProxy('%s/xmlrpc/2/object' % server_url)

if user_id:

    search_domain = []
    res_partner_objects = models.execute_kw(db_name, user_id, password,
                                            'res.partner', 'search_read',
                                            [search_domain, res_partner_fields],
                                            {}
                                           )
    res_partner = pd.DataFrame(res_partner_objects)

    print(res_partner)

    conn = sqlite3.connect('data/jcafb_2025.db')

    cur = conn.cursor()
    cur.execute('DELETE FROM res_partner')
    conn.commit()

    # res_partner.to_sql('res_partner', conn, if_exists='replace', index=False)
    res_partner.to_sql('res_partner', conn, if_exists='append', index=False)

    conn.close()


        id                        name             type
0     3931          Abel Lucas Bertoni  clv.patient_rec
1     3481          Abel Lucas Bertoni      clv.patient
2     3932          Adalberto de Sousa  clv.patient_rec
3     3496          Adalberto de Sousa      clv.patient
4     3933  Adelaide Francisco Machado  clv.patient_rec
...    ...                         ...              ...
1942  3464      Zetir Maria Souza Tosi      clv.patient
1943  4772        Zilda Pereira Cruaia  clv.patient_rec
1944  3632        Zilda Pereira Cruaia      clv.patient
1945  4773       Zilda Teixeira Cintra  clv.patient_rec
1946  3924       Zilda Teixeira Cintra      clv.patient

[1947 rows x 3 columns]


## Importação dos dados de "res.company"

In [6]:
res_company_fields = ['id', 'name']

common = client.ServerProxy('%s/xmlrpc/2/common' % server_url)
user_id = common.authenticate(db_name, username, password, {})
models = client.ServerProxy('%s/xmlrpc/2/object' % server_url)

if user_id:

    search_domain = []
    res_company_objects = models.execute_kw(db_name, user_id, password,
                                            'res.company', 'search_read',
                                            [search_domain, res_company_fields],
                                            {}
                                           )
    res_company = pd.DataFrame(res_company_objects)

    print(res_company)

    conn = sqlite3.connect('data/jcafb_2025.db')

    cur = conn.cursor()
    cur.execute('DELETE FROM res_company')
    conn.commit()

    # res_company.to_sql('res_company', conn, if_exists='replace', index=False)
    res_company.to_sql('res_company', conn, if_exists='append', index=False)

    conn.close()


   id             name
0   1  CLVhealth-JCAFB


## Importação dos dados de "res.users"

In [7]:
# res_users_fields = ['id', 'name', 'partner_id', 'company_id', 'parent_id', 'tz', 'lang', 'country_id',
#                     'login', 'password', 'groups_id', 'active', 'image_1920']
res_users_fields = ['id', 'name', 'partner_id', 'company_id', 'parent_id', 'tz', 'lang', 'country_id',
                    'login', 'password', 'active', 'image_1920']

common = client.ServerProxy('%s/xmlrpc/2/common' % server_url)
user_id = common.authenticate(db_name, username, password, {})
models = client.ServerProxy('%s/xmlrpc/2/object' % server_url)

if user_id:

    search_domain = []
    res_users_objects = models.execute_kw(db_name, user_id, password,
                                          'res.users', 'search_read',
                                          [search_domain, res_users_fields],
                                          {}
                                         )
    res_users = pd.DataFrame(res_users_objects)

    for i, row in res_users.iterrows():

        if row['partner_id']:
            res_users['partner_id'].values[i] = row['partner_id'][0]
        else:
            res_users['partner_id'].values[i] = None

        if row['company_id']:
            res_users['company_id'].values[i] = row['company_id'][0]
        else:
            res_users['company_id'].values[i] = None

        if row['parent_id']:
            res_users['parent_id'].values[i] = row['parent_id'][0]
        else:
            res_users['parent_id'].values[i] = None

        if row['country_id']:
            res_users['country_id'].values[i] = row['country_id'][0]
        else:
            res_users['country_id'].values[i] = None

    print(res_users)

    conn = sqlite3.connect('data/jcafb_2025.db')

    cur = conn.cursor()
    cur.execute('DELETE FROM res_users')
    conn.commit()

    # res_users.to_sql('res_users', conn, if_exists='replace', index=False)
    res_users.to_sql('res_users', conn, if_exists='append', index=False)

    conn.close()


      id                              name partner_id company_id parent_id  \
0      2                     Administrator          3          1      None   
1      8           Alex Monteiro Magalhães         10          1         1   
2      9               Alexandre Bakanovas         11          1         1   
3     10         Alice Herminia Serpentino         12          1         1   
4     11                Alice Mota Almeida         13          1         1   
..   ...                               ...        ...        ...       ...   
254  225    Vitória Alves da Silva Varollo        227          1         1   
255  226         Vitória Monteiro Bernardo        228          1         1   
256  227       Wallace Androm Gomez Junior        229          1         1   
257  228  Yasmin Catharine da Paz Mendonça        230          1         1   
258  229     Zaira Maria De Sousa Pinheiro        231          1         1   

                    tz   lang country_id                       

## Importação dos dados de "clv_phase"

In [8]:
clv_phase_fields = ['id', 'name', 'description']

common = client.ServerProxy('%s/xmlrpc/2/common' % server_url)
user_id = common.authenticate(db_name, username, password, {})
models = client.ServerProxy('%s/xmlrpc/2/object' % server_url)

if user_id:

    search_domain = []
    clv_phase_objects = models.execute_kw(db_name, user_id, password,
                                          'clv.phase', 'search_read',
                                          [search_domain, clv_phase_fields],
                                          {}
                                         )
    clv_phase = pd.DataFrame(clv_phase_objects)

    for i, row in clv_phase.iterrows():

        if row['description'] is False:
            clv_phase['description'].values[i] = None

    print(clv_phase)

    conn = sqlite3.connect('data/jcafb_2025.db')

    cur = conn.cursor()
    cur.execute('DELETE FROM clv_phase')
    conn.commit()

    # clv_phase.to_sql('clv_phase', conn, if_exists='replace', index=False)
    clv_phase.to_sql('clv_phase', conn, if_exists='append', index=False)

    conn.close()


   id         name                                        description
0   1   JCAFB-2017                                               None
1   2   JCAFB-2018                                               None
2   3   JCAFB-2019                                               None
3   4   JCAFB-2020                                               None
4   5  JCAFB-2021v  Jornada Virtual simulando uma quinta visita à ...
5   6  JCAFB-2022v  Jornada Virtual simulando uma visita a uma Com...
6   7   JCAFB-2023       Primeiro ano do ciclo de Itapirapuã Paulista
7   8   JCAFB-2024        Segundo ano do ciclo de Itapirapuã Paulista
8   9   JCAFB-2025                      Primeiro ano do ciclo de Avaí


## Importação dos dados de "clv_patient"

In [10]:
clv_patient_fields = ['id', 'name', 'code', 'gender', 'birthday', 'phase_id',
                      'address_name', 'street', 'street_name', 'street_number', 'street2', 'street_number2',
                      'zip', 'city_id', 'country_id', 'mobile', 'email', 'marker_ids'
                     ]

common = client.ServerProxy('%s/xmlrpc/2/common' % server_url)
user_id = common.authenticate(db_name, username, password, {})
models = client.ServerProxy('%s/xmlrpc/2/object' % server_url)

if user_id:

    search_domain = []
    clv_patient_objects = models.execute_kw(db_name, user_id, password,
                                            'clv.patient', 'search_read',
                                            [search_domain, clv_patient_fields],
                                            {}
                                           )
    clv_patient = pd.DataFrame(clv_patient_objects)

    for i, row in clv_patient.iterrows():

        if row['city_id']:
            clv_patient['phase_id'].values[i] = row['phase_id'][0]
        else:
            clv_patient['phase_id'].values[i] = None

        if row['city_id']:
            clv_patient['city_id'].values[i] = row['city_id'][0]
        else:
            clv_patient['city_id'].values[i] = None

        if row['country_id']:
            clv_patient['country_id'].values[i] = row['country_id'][0]
        else:
            clv_patient['country_id'].values[i] = None

        if row['marker_ids']:
            clv_patient['marker_ids'].values[i] = str(row['marker_ids'])
        else:
            clv_patient['marker_ids'].values[i] = None

    print(clv_patient)

    conn = sqlite3.connect('data/jcafb_2025.db')

    cur = conn.cursor()
    cur.execute('DELETE FROM clv_patient')
    conn.commit()

    # clv_patiBaseExceptionent.to_sql('clv_patient', conn, if_exists='replace', index=False)
    clv_patient.to_sql('clv_patient', conn, if_exists='append', index=False)

    conn.close()


       id                            name            code gender    birthday  \
0    1224              Abel Lucas Bertoni  709.052.078-15      M  1955-06-01   
1    1239              Adalberto de Sousa  826.257.608-00      M  1955-12-12   
2    1467      Adelaide Francisco Machado  280.197.278-92      M  1948-03-13   
3    1414        Adelia Aparecida Fontana  058.521.998-26      F  1964-08-29   
4    1247          Ademar Antonio Dagoano  042.270.498-90      M  1947-05-14   
..    ...                             ...             ...    ...         ...   
838   918      Zeni Maria de Souza Pivato  096.142.708-61      F  1946-12-14   
839  1077  Zenilce de Souza Lima da Silva  347.004.648-44      F  1947-08-20   
840  1207          Zetir Maria Souza Tosi  825.869.938-53      F  1949-11-18   
841  1375            Zilda Pereira Cruaia  257.321.268-80      F  1957-10-04   
842  1667           Zilda Teixeira Cintra  049.801.878-46      F  1956-07-30   

    phase_id                           